In [ ]:
# |export
#|default_exp p00_titanic_from_scratch


In [ ]:
# |export
import os
import time
import pathlib
import argparse
import torch
from torch import tensor



In [ ]:
# |export
start_time=time.time()
debug=True
_code_git_version="813fa8922c471c2496250f275b2f04e071915191"
_code_repository="https://github.com/plops/cl-py-generator/tree/master/example/96_colab_fastai/source/"
_code_generation_time="23:14:29 of Saturday, 2022-08-27 (GMT+1)"
start_time=time.time()
debug=True


In [ ]:
# |export
parser=argparse.ArgumentParser()
parser.add_argument("-v", "--verbose", help="enable verbose output", action="store_true")
args=parser.parse_args()


In [ ]:
# |export
path=pathlib.Path("titanic")
if ( not(path.exists()) ):
    import zipfile
    import kaggle
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f"{path}.zip").extractall(path)


In [ ]:
# |export
import torch
import numpy as np
import pandas as pd
line_char_width=140
np.set_print_options(linewidth=line_char_width)
torch.set_print_options(linewidth=line_char_width, sci_mode=False, edgeitems=7)
pd.set_option("display_width", line_char_width)


In [ ]:
# |export
df=pd.read_csv(((path)/("train.csv")))
df


In [ ]:
df.isna().sum()


In [ ]:
# |export
mode=df.mode().iloc[0]


In [ ]:
# |export
df.fillna(modes, inplace=True)


In [ ]:
df.isna().sum()


In [ ]:
df.describe(include=(np.number,))


In [ ]:
df.Fare.hist()


In [ ]:
# |export
df["LogFare"]=np.log(((1)+(df.Fare)))


In [ ]:
# |export
df=pd.get_dummies(df, columns=["Sex", "Pclass", "Embarked"])
df.columns


In [ ]:
# |export
t_dep=tensor(df.Survived)
